<a href="https://colab.research.google.com/github/prisar/ai_notebooks/blob/main/nb_067.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### paper

Learning under Latent Group Sparsity
via Diffusion on Networks

[link](https://arxiv.org/pdf/2507.15097)

In [ ]:
import numpy as np

def estimate_graph_laplacian(X, quantile=0.75):
    """
    Estimates the graph Laplacian from the data matrix X.
    Corresponds to Algorithm 6 in the paper. [cite: 246]
    """
    # Ensure X is centered for correlation calculation
    X_centered = X - X.mean(axis=0)

    # 1. Estimate the correlation matrix
    p = X.shape[1]
    if p == 0:
        return np.array([[]])
    R_hat = np.corrcoef(X_centered, rowvar=False)
    # Handle case where a column has zero variance
    R_hat = np.nan_to_num(R_hat)

    # 2. Threshold the correlation matrix to get the adjacency matrix
    # [cite: 244]
    abs_R_hat = np.abs(R_hat)
    # Exclude the diagonal from quantile calculation
    off_diagonal_indices = ~np.eye(p, dtype=bool)
    threshold = np.quantile(abs_R_hat[off_diagonal_indices], quantile)
    A = (abs_R_hat > threshold).astype(int)
    np.fill_diagonal(A, 0) # No self-loops

    # 3. Compute the unnormalized graph Laplacian
    #
    D = np.diag(np.sum(A, axis=1))
    L = D - A

    return L, A

In [ ]:
# Create some example data
X_example = np.random.rand(100, 10) # 100 samples, 10 features

# Call the function with the example data
L_example, A_example = estimate_graph_laplacian(X_example)

# Display the resulting Laplacian and Adjacency matrices
print("Estimated Graph Laplacian (L):")
display(L_example)

print("\nEstimated Adjacency Matrix (A):")
display(A_example)

Estimated Graph Laplacian (L):


array([[ 1,  0,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  3,  0, -1,  0, -1,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1, -1,  0,  5, -1, -1,  0,  0,  0, -1],
       [ 0,  0,  0, -1,  1,  0,  0,  0,  0,  0],
       [ 0, -1,  0, -1,  0,  4, -1,  0,  0, -1],
       [ 0,  0,  0,  0,  0, -1,  1,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  2,  0, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1, -1],
       [ 0,  0,  0, -1,  0, -1,  0, -1, -1,  4]])


Estimated Adjacency Matrix (A):


array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 0, 1, 1, 0]])

In [ ]:
def simulate_heat_flow(A, t_flow, B):
    """
    Simulates continuous-time random walks to generate the heat flow matrix H.
    Corresponds to Algorithm 1 in the paper. [cite: 179]
    """
    p = A.shape[0]
    H = np.zeros((p, B), dtype=int)
    degrees = np.sum(A, axis=1)
    neighbors = [np.where(A[i] > 0)[0] for i in range(p)]

    for i in range(p): # For each starting vertex
        for j in range(B): # For each simulation
            current_node = i
            current_time = 0.0

            while current_time < t_flow:
                node_degree = degrees[current_node]
                if node_degree == 0:
                    break # Isolated node

                # Time to next jump is Exponential(rate=degree)
                # [cite: 628]
                time_step = np.random.exponential(1.0 / node_degree)
                current_time += time_step

                if current_time < t_flow:
                    # Jump to a random neighbor
                    # [cite: 630]
                    current_node = np.random.choice(neighbors[current_node])

            H[i, j] = current_node
    return H

def apply_heat_flow(H, f):
    """
    Estimates the action of the heat flow operator e^(-tL) on a vector f.
    Corresponds to Algorithm 2 in the paper. [cite: 179]
    """
    p = H.shape[0]
    g = np.zeros(p)

    for i in range(p):
        # Endpoints of B walks started at i
        endpoints = H[i, :]
        # Average the values of f at these endpoints
        # [cite: 160, 179]
        g[i] = np.mean(f[endpoints])

    return g